In [8]:
#   Ejercicio 3

from pymongo import MongoClient
import json

client = MongoClient("mongodb://mongo:27017")

def save_to_txt(filename, data):
    with open(filename, "w", encoding="utf-8") as f:
        for doc in data:
            f.write(json.dumps(doc, default=str, indent=2) + "\n")

db = client["mi_base"]
proveedores = db["proveedores"]

query3 = proveedores.aggregate([
    {"$unwind": "$telefonos"},
    {"$project": {
        "_id": 0,
        "razon_social": 1,
        "CUIT": 1,
        "telefono": "$telefonos"
    }}
])
save_to_txt("query3.txt", list(query3))

In [9]:
#   Ejercicio 6

from pymongo import MongoClient
import json


client = MongoClient("mongodb://mongo:27017")

def save_to_txt(filename, data):
    with open(filename, "w", encoding="utf-8") as f:
        for doc in data:
            f.write(json.dumps(doc, default=str, indent=2) + "\n")

db = client["mi_base"]
proveedores = db["proveedores"]

query6 = proveedores.aggregate([
    {"$lookup": {
        "from": "pedidos",
        "localField": "_id",
        "foreignField": "id_proveedor",
        "as": "ordenes"
    }},
    {"$project": {
        "_id": 0,
        "id_proveedor": "$_id",
        "razon_social": 1,
        "total_ordenes": {"$size": "$ordenes"},
        "monto_total_sin_iva": {
            "$sum": {"$map": {
                "input": "$ordenes",
                "as": "orden",
                "in": "$$orden.total_sin_iva"
            }}
        },
        "monto_total_con_iva": {
            "$sum": {
                "$map": {
                    "input": "$ordenes",
                    "as": "orden",
                    "in": {
                        "$add": ["$$orden.total_sin_iva", "$$orden.iva"]
                    }
                }
            }
        }
    }}
])
save_to_txt("query6.txt", list(query6))

In [10]:
#   Ejercicio 9

from pymongo import MongoClient
import json

client = MongoClient("mongodb://mongo:27017")
db = client["mi_base"]
pedidos = db["pedidos"]

def save_to_txt(filename, data):
    with open(filename, "w", encoding="utf-8") as f:
        for doc in data:
            f.write(json.dumps(doc, default=str, indent=2) + "\n")

query9 = pedidos.aggregate([
    {"$unwind": "$detalle"},
    {"$lookup": {
        "from": "productos",
        "localField": "detalle.id_producto",
        "foreignField": "_id",
        "as": "producto"
    }},
    {"$unwind": "$producto"},
    {"$match": {
        "producto.marca": "COTO"
    }},
    {"$project": {
        "_id": 1,
        "fecha": 1,
        "id_proveedor": 1,
        "total_sin_iva": 1,
        "iva": 1,
        "detalle": 1,
        "producto": 1
    }}
])
save_to_txt("query9.txt", list(query9))


In [11]:
#   Ejercicio 12

from pymongo import MongoClient
import json

client = MongoClient("mongodb://mongo:27017")
db = client["mi_base"]

def save_to_txt(filename, data):
    with open(filename, "w", encoding="utf-8") as f:
        for doc in data:
            f.write(json.dumps(doc, default=str, indent=2) + "\n")

db.command({
    "create": "proveedores_inhabilitados_activos",
    "viewOn": "proveedores",
    "pipeline": [
        {"$match": {
            "activo": True,
            "habilitado": False
        }},
        {"$project": {
            "_id": 0,
            "id_proveedor": "$_id",
            "razon_social": 1,
            "CUIT": 1,
            "direccion": 1,
            "tipo_sociedad": 1
        }}
    ]
})


## info de la vista en query12.txt
vista = db["proveedores_inhabilitados_activos"]
rta = list(vista.find())
save_to_txt("query12.txt", rta)


In [15]:
#   Ejercicio 15

from pymongo import MongoClient
from datetime import datetime
import json

client = MongoClient("mongodb://mongo:27017")
db = client["mi_base"]
proveedores = db["proveedores"]

## data de la orden 
nueva_orden = {
    "_id": 364,
    "id_proveedor": 1,
    "fecha": datetime.strptime("2025-06-19", "%Y-%m-%d"),
    "total_sin_iva": 1000.0,
    "iva": 210.0,
    "detalle": [
        {"id_producto": 1, "nro_item": 1, "cantidad": 3}
    ]
}

if proveedores.find_one({"_id": nueva_orden["id_proveedor"]}):
    pedidos.insert_one(nueva_orden)
    with open("query15.txt", "w") as f:
        f.write("Nueva orden registrada:\n")
        f.write(json.dumps(nueva_orden, default=str, indent=2))
else:
    with open("query15.txt", "w") as f:
        f.write(f"No se pudo registrar orden: proveedor con ID {nueva_orden['id_proveedor']} no existe.\n")

DuplicateKeyError: E11000 duplicate key error collection: mi_base.pedidos index: _id_ dup key: { _id: 364 }, full error: {'index': 0, 'code': 11000, 'errmsg': 'E11000 duplicate key error collection: mi_base.pedidos index: _id_ dup key: { _id: 364 }', 'keyPattern': {'_id': 1}, 'keyValue': {'_id': 364}}